## Code Information
<small> <quote>
**Author:**  Shafagh Keyvanian [shkey@seas.upenn.edu]  <br>
**Date**: *Spring 2024*
</quote> </small>

<Large> <quote>
**All Subjects**<br>
**Converts Motive export .csv to Upper-arm Euler angles**<br>
**Bones: Hip, Ab, Chest, Shoulder, UArm, FArm, Hand** </quote> </Large>

Input:  
OptiTrack data: Motive exported .csv file
- Folder: 
<small> <blockquote> ../data/raw_exports/ </blockquote> </small>
- Skeleton:  
<small> <blockquote>
        Skeleton: Conventional Upper (27 Markers) - 43 Bones<br>
        Gaps manually filled with "Linear Interpolation"<br>
        Export: Bones- position & quaternion, Markers- position </blockquote> </small>

Output:  
.npz file containing Upper-body Euler angles
- Folder: 
<small> <blockquote> ../data/prossesed_exports/ </blockquote> </small>
- Columns:  
<small> <blockquote> 'time', 'eul_right', 'eul_left', 'eul_head' </blockquote> </small>
- Chains:  
<small> <blockquote>
        chain_R = ['Skeleton', 'Ab', 'Chest', 'RShoulder', 'RUArm','RFArm','RHand']<br>
        chain_L = ['Skeleton', 'Ab', 'Chest', 'LShoulder', 'LUArm','LFArm','LHand']<br>
        chain_H = ['Skeleton', 'Ab', 'Chest', 'Neck', 'Head'] </blockquote> </small>

In [ ]:
#%% import libraries
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import Axes3D
from tf.transformations import quaternion_matrix, euler_from_matrix, euler_matrix

In [ ]:
# # Extract Euler angles from raw data for all files in the folder
# Make two chains for right and left upper limb 
chain_R = ['Skeleton', 'Ab', 'Chest','RShoulder','RUArm','RFArm','RHand']
chain_L = ['Skeleton', 'Ab', 'Chest','LShoulder','LUArm','LFArm','LHand']
chain_H = ['Skeleton', 'Ab', 'Chest','Neck', 'Head']

init_folder = '../data/raw_exports/'     
csv_files = [file for file in os.listdir(init_folder) if file.endswith('.csv')]
print('Files to process:', csv_files.__len__())

for i, filename in enumerate(csv_files):
    csvfile = str(os.path.join(init_folder, filename))
    print(csvfile)

    raw_data = pd.read_csv(csvfile, header=[3, 5, 6], index_col=0)
    raw_data.rename(columns=lambda x: x.replace('Skeleton:', ''), level=0, inplace=True)
    time = pd.read_csv(csvfile, header=6, usecols=[1]).values 
    skl_R, skl_L, skl_H = raw_data[chain_R].copy(), raw_data[chain_L].copy(), raw_data[chain_H].copy()

    # Get HUB skeleton dimensions
    t_test = int(len(time)/2)
    # Offset angle between strnm_dir_R/L and strnm_dir_H
    strnm_dir_R = skl_R[('RShoulder', 'Position')].iloc[t_test] - skl_R[('Chest', 'Position')].iloc[t_test]
    strnm_dir_L = skl_L[('LShoulder', 'Position')].iloc[t_test] - skl_L[('Chest', 'Position')].iloc[t_test]
    strnm_dir_H = skl_H[('Neck', 'Position')].iloc[t_test] - skl_H[('Chest', 'Position')].iloc[t_test]
    strnm_ang_R = np.arccos(np.dot(strnm_dir_R, strnm_dir_H)/(np.linalg.norm(strnm_dir_R)*np.linalg.norm(strnm_dir_H)))
    strnm_ang_L = np.arccos(np.dot(strnm_dir_L, strnm_dir_H)/(np.linalg.norm(strnm_dir_L)*np.linalg.norm(strnm_dir_H)))

    def get_euler_HUB(chain, t=t_test):  
        skl = skl_R if chain==chain_R else skl_L if chain==chain_L else skl_H
        eul = []
        R_prox = np.eye(3)
        for j in range(len(chain)):
            # print(j, chain[j], skl[(chain[j], 'Rotation')].iloc[t])
            q = skl[(chain[j], 'Rotation')].iloc[t].values
            # print('q', q)
            R_global = quaternion_matrix(q)[:3,:3]
            R_rel = R_prox.T @ R_global
            R_prox = R_global
            if chain==chain_H:
                eul_sequnce = 'ryxz'
            else:
                eul_sequnce = 'ryxz' if j < 3 else 'rzyx'
            eul_values = euler_from_matrix(R_rel, axes=eul_sequnce)

            if chain==chain_R and j>2:
                eul_values = [eul_values[0], -eul_values[1], eul_values[2]]

            if chain==chain_L and j>2:
                # eul_values = [-eul_values[0], -eul_values[1], eul_values[2]]
                eul_values = [-eul_values[0], eul_values[1], eul_values[2]]
                
            eul.append(eul_values)
        return np.array(eul)

    # Extract euler angles each chain for all time frames
    def extract_angles_all(chain):
        eul_all = np.zeros((len(time), len(chain), 3))
        for t in range(len(time)):
            eul_all[t] = get_euler_HUB(chain, t)
        return eul_all

    eul_right = extract_angles_all(chain_R)
    eul_left = extract_angles_all(chain_L)
    eul_head = extract_angles_all(chain_H)

    # Save Euler angles to npz file with automatic naming
    npz_file = '../data/HUB_angles/' + filename.replace('.csv', '.angles_HUB.npz')
    np.savez(npz_file, time=time, eul_right=eul_right, eul_left=eul_left, eul_head=eul_head)



In [ ]:
# #%% All subjects Extract Euler angles from quaternions

# # Subjects
# n_subjects = 5
# subject_labels = ['subject0', 'subject1', 'subject2', 'subject3', 'subject4']
# healthy_arm  = ['R', 'L', 'L', 'L', 'R']

# # Define chains and DOFs
# # Make two chains for right and left upper limb 
# chain_R = ['Skeleton', 'Ab', 'Chest','RShoulder','RUArm','RFArm','RHand']
# chain_L = ['Skeleton', 'Ab', 'Chest','LShoulder','LUArm','LFArm','LHand']
# chain_H = ['Skeleton', 'Ab', 'Chest','Neck', 'Head']
 
# dof_names = ['Clavicle Protraction', 'Clavicle Elevation', 'Clavicle Axial Rotation', \
#              'Shoulder Abduction', 'Shoulder Rotation', 'Shoulder Extension', 'Elbow Flexion', \
#              'Elbow Supination', 'Wrist Deviation', 'Wrist Supination', 'Wrist Flexion']
# dof_codes = ['cl_x', 'cl_y', 'cl_z', 'sh_abd', 'sh_rot', 'sh_ext', 'el_fle', 'el_sup', 'wr_dev', 'wr_sup', 'wr_fle']

# n_subjects = 5
# subject_labels = ['Subject 0', 'Subject 1', 'Subject 2', 'Subject 3', 'Subject 4', 'All Subjects']

# HUB_dof_R = ['pelvis_tilt', 'pelvis_list', 'pelvis_rotation',   'Ab_bending', 'Ab_rotation', 'Ab_twist',    'Chest_bending', 'Chest_rotation', 'Chest_twist', \
#              'RClavicle Protraction', 'RClavicle Elevation','RClavicle Axial Rotation',     'Shoulder Abduction', 'Shoulder Rotation', 'Shoulder Extension', \
#              'Elbow Flexion', 'Elbow Supination', 'Elbow_0',    'Wrist Deviation', 'Wrist Supination', 'Wrist Flexion']

# HUB_dof_L = ['pelvis_tilt', 'pelvis_list', 'pelvis_rotation',   'Ab_bending', 'Ab_rotation', 'Ab_twist',    'Chest_bending', 'Chest_rotation', 'Chest_twist', \
#              'LClavicle Protraction', 'LClavicle Elevation', 'LClavicle Axial Rotation',    'Shoulder Abduction', 'Shoulder Rotation', 'Shoulder Extension', \
#              'Elbow Flexion', 'Elbow Supination', 'Elbow_0',    'Wrist Deviation', 'Wrist Supination', 'Wrist Flexion']
             
# HUB_dof_H = ['pelvis_tilt', 'pelvis_list', 'pelvis_rotation',   'Ab_bending', 'Ab_rotation', 'Ab_twist',    'Chest_bending', 'Chest_rotation', 'Chest_twist', \
#              'Neck Bending', 'Neck Rotation', 'Neck Twist',     'Head Flexion', 'Head Rotation', 'Head Twist']

# def load_raw_data(s):
#     side = healthy_arm[s]
#     folder = '../data/raw_exports/'     
#     csv_files = [file for file in os.listdir(folder) if f'subject{s}_{side}h_' in file and file.endswith('.csv')]
#     print(csv_files)

#     # time, skl_R, skl_L, skl_H =  np.array([[0]]), pd.DataFrame(), pd.DataFrame(), pd.DataFrame() #np.array([[0]]),
#     for i, file in enumerate(csv_files):
#         csvfile = str(os.path.join(folder, file))
#         print(csvfile)
#         raw_data = pd.read_csv(csvfile, header=[3, 5, 6], index_col=0)
#         raw_data.rename(columns=lambda x: x.replace('Skeleton:', ''), level=0, inplace=True)
#         time_i = pd.read_csv(csvfile, header=6, usecols=[1]).values 
#         skl_R_i, skl_L_i, skl_H_i = raw_data[chain_R].copy(), raw_data[chain_L].copy(), raw_data[chain_H].copy()

#         if i == 0:
#             time, skl_R, skl_L, skl_H = time_i, skl_R_i, skl_L_i, skl_H_i
#         else:
#             time = np.concatenate((time, time_i+time[-1]+0.01), axis=0).flatten()
#             skl_R = pd.concat([skl_R, skl_R_i], axis=0, ignore_index=True)
#             skl_L = pd.concat([skl_L, skl_L_i], axis=0, ignore_index=True)
#             skl_H = pd.concat([skl_H, skl_H_i], axis=0, ignore_index=True)
        
#     return time, skl_R, skl_L, skl_H

# for subject in range(n_subjects):
#     print(f'Extracting Euler angles for subject {subject}')
#     side = healthy_arm[subject]
#     time, skl_R, skl_L, skl_H = load_raw_data(subject)
#     # Get HUB skeleton dimensions
#     t_test = 5000
#     # Offset angle between strnm_dir_R/L and strnm_dir_H
#     # strnm_dir_R = np.array(skl_R.loc[t_test, ('RShoulder', 'Position')]) - np.array(skl_R.loc[t_test, ('Chest', 'Position')])
#     strnm_dir_R = skl_R[('RShoulder', 'Position')].iloc[t_test] - skl_R[('Chest', 'Position')].iloc[t_test]
#     strnm_dir_L = skl_L[('LShoulder', 'Position')].iloc[t_test] - skl_L[('Chest', 'Position')].iloc[t_test]
#     strnm_dir_H = skl_H[('Neck', 'Position')].iloc[t_test] - skl_H[('Chest', 'Position')].iloc[t_test]
#     strnm_ang_R = np.arccos(np.dot(strnm_dir_R, strnm_dir_H)/(np.linalg.norm(strnm_dir_R)*np.linalg.norm(strnm_dir_H)))
#     strnm_ang_L = np.arccos(np.dot(strnm_dir_L, strnm_dir_H)/(np.linalg.norm(strnm_dir_L)*np.linalg.norm(strnm_dir_H)))

#     def get_euler_HUB(chain, t=t_test):  
#         skl = skl_R if chain==chain_R else skl_L if chain==chain_L else skl_H
#         eul = []
#         R_prox = np.eye(3)
#         for j in range(len(chain)):
#             # print(j, chain[j], skl[(chain[j], 'Rotation')].iloc[t])
#             q = skl[(chain[j], 'Rotation')].iloc[t].values
#             # print('q', q)
#             R_global = quaternion_matrix(q)[:3,:3]
#             R_rel = R_prox.T @ R_global
#             R_prox = R_global
#             if chain==chain_H:
#                 eul_sequnce = 'ryxz'
#             else:
#                 eul_sequnce = 'ryxz' if j < 3 else 'rzyx'
#             eul_values = euler_from_matrix(R_rel, axes=eul_sequnce)

#             if chain==chain_R and j>2:
#                 eul_values = [eul_values[0], -eul_values[1], eul_values[2]]

#             if chain==chain_L and j>2:
#                 # eul_values = [-eul_values[0], -eul_values[1], eul_values[2]]
#                 eul_values = [-eul_values[0], eul_values[1], eul_values[2]]
                
#             eul.append(eul_values)
#         return np.array(eul)

#     # Extract euler angles each chain for all time frames
#     def extract_angles_all(chain):
#         eul_all = np.zeros((len(time), len(chain), 3))
#         for t in range(len(time)):
#             eul_all[t] = get_euler_HUB(chain, t)
#         # print(chain, eul_all.shape)
#         return eul_all

#     eul_right = extract_angles_all(chain_R)
#     eul_left = extract_angles_all(chain_L)
#     eul_head = extract_angles_all(chain_H)

#     # Save Euler angles to npz file with automatic naming
#     # npz_file = '../data/HUB_angles/subject'+str(subject)+'_HUB_angles.npz'
#     npz_file = f'../data/HUB_angles/subject{subject}_{side}h_HUB_angles.npz'
#     np.savez(npz_file, time=time, eul_right=eul_right, eul_left=eul_left, eul_head=eul_head)

#     print(f'Saved Euler angles to {npz_file}')

In [ ]:
#%% Visualization of joint locations at time t or with user-defined angles
Motive = True
t = 5000

# Initialize Skeleton Angles
angles_init_base = [[0, 0, 0],  # 'pelvis_tilt', 'pelvis_list', 'pelvis_rotation'
                   [0, 0, 0],  # 'Ab_bending', 'Ab_extension', 'Ab_twist'
                   [0, 0, 0]]  # 'Chest_bending', 'Chest_extension', 'Chest_twist'

angles_init_L = np.vstack([angles_init_base,
                [[0, 0, 0],     # LClavicle (Protraction, Elevation, Rotation) --- (abduction, elevation, upward_rot)
                 [0, 0, 0],    # 'Shoulder Abduction', 'Shoulder Rotation', 'Shoulder Extension'
                 [90, 0, 0],    # 'Elbow Flexion', 'Elbow_0', 'Elbow Supination'
                 [0, 0, 0]]])   # 'Wrist Flexion', 'Wrist_0', 'Wrist Deviation' 

angles_init_R = angles_init_L

angles_init_H = np.vstack([angles_init_base,
                [[0, 0, 0],      # 'Neck Bending', 'Neck Rotation', 'Neck Twist'    
                 [0, 0, 0]]])    # 'Head Flexion', 'Head Rotation', 'Head Twist'

def get_skl_angles(chain):
    if Motive:
        skl = skl_R if chain==chain_R else skl_L if chain==chain_L else skl_H
        HUB_angles = get_euler_HUB(chain)
    else:
        HUB_angles = angles_init_R if chain==chain_R else angles_init_L if chain==chain_L else angles_init_H
        HUB_angles = np.array(HUB_angles) * np.pi/180
    return HUB_angles

def get_pos_HUB_raw(chain, t):  
    skl = skl_R if chain==chain_R else skl_L if chain==chain_L else skl_H
    positions = []
    for j in range(len(chain)):
        pos = skl[(chain[j], 'Position')].iloc[t]
        positions.append(np.array(pos))
    positions -= positions[0]
    return positions

def get_pos_HUB_FK(chain): 
    positions = get_pos_HUB_raw(chain)
    bone_length = np.linalg.norm(np.diff(positions, axis=0), axis=1)

    eul_angles = get_skl_angles(chain)    
    fk_pose = np.zeros((len(eul_angles), 3))
    pos_prox = np.zeros(3)
    R_prox = np.eye(3)
    for i in range(3):
        R_rel = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes='ryxz')[:3, :3]
        R_prox = R_prox @ R_rel
        if i==2:
            offset_ang = strnm_ang_R if chain==chain_R else -strnm_ang_L if chain==chain_L else 0
            R_off = euler_matrix(0, 0, offset_ang, axes='rzxy')[:3, :3]
            pos_prox += R_prox @ R_off @ np.array([0, 0, bone_length[i]])
        else:
            pos_prox += R_prox @ np.array([0, 0, bone_length[i]])
        fk_pose[i+1] = pos_prox
    for i in range(3, len(eul_angles)-1):
        if chain==chain_R:  
            # R_rel = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes='rzyx')[:3, :3]
            R_rel = euler_matrix(eul_angles[i, 0], -eul_angles[i, 1], eul_angles[i, 2], axes='rzyx')[:3, :3]
            R_prox = R_prox @ R_rel
            pos_prox += R_prox @ np.array([bone_length[i], 0, 0])
        elif chain==chain_L:
            # R_rel = euler_matrix(-eul_angles[i, 0], -eul_angles[i, 1], eul_angles[i, 2], axes='rzyx')[:3, :3]
            R_rel = euler_matrix(-eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes='rzyx')[:3, :3]
            R_prox = R_prox @ R_rel
            pos_prox += R_prox @ np.array([-bone_length[i], 0, 0])
        elif chain==chain_H:
            R_rel = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes='ryxz')[:3, :3]
            R_prox = R_prox @ R_rel
            pos_prox += R_prox @ np.array([0, 0, bone_length[i]])
        fk_pose[i+1] = pos_prox
    return fk_pose

def print_joint_properties(chain):
    print("{:<10} {:<25} {:<25} {:<25}".format("Joint", "Raw", "FK", "Euler Angles"))
    for i in range(len(chain)):
        print("{:<10} {:<25} {:<25} {:<25}".format(chain[i], str(np.around(get_pos_HUB_raw(chain)[i], 2)), str(np.around(get_pos_HUB_FK(chain)[i], 2)), str(np.around(get_skl_angles(chain)[i], 2)) ))

def visualize_joint_locations():   
    colors_R = ['black', 'orange', 'navy', 'cyan', 'red', 'green', 'blue']
    colors_L = ['black', 'orange', 'navy', 'purple', 'magenta', 'lime', 'teal']
    colors_H = ['black', 'orange', 'navy', 'black', 'black']

    n_ax = 2      
    fig_title = 'HUB Skeleton: FK vs. Raw' 
    ax_title = ['COMETH Forward Kinematics', 'HUB Forward Kinematics', 'Motive Raw Positions']
    joint_marker = ['o', '*', 'v'] 

    fig, axes = plt.subplots(1, n_ax, subplot_kw={'projection': '3d'}, figsize=(n_ax*5-1, 4), constrained_layout=True, num='Skeleton Visualization')
    fig.suptitle(fig_title, fontsize='xx-large', y=1.05)#, fontweight='bold')
    for chain in [chain_R, chain_L, chain_H]:
        joint_colors = colors_R if chain==chain_R else colors_L if chain==chain_L else colors_H
        pos = [get_pos_HUB_FK(chain), get_pos_HUB_raw(chain)] 
        
        for i in range(n_ax):
            axes[i].set_title(ax_title[i], fontsize='large', pad=30, fontweight='bold')
            # axes[i].view_init(elev=25, azim=160)
            axes[i].view_init(elev=35, azim=150)
            axes[i].set(xlabel='X', ylabel='Y', zlabel='Z', xlim=(-500, 500), ylim=(-500, 500), zlim=(-100, 900))
            axes[i].plot([0, 0], [0, 0], [0, 800], 'k--')
            axes[i].plot([0, 0], [0, 500], [0, 0], 'k--')
            axes[i].plot([0, 500], [0, 0], [0, 0], 'k--')
            for j in range(len(chain)):
                axes[i].scatter(pos[i][j][0], pos[i][j][1], pos[i][j][2], marker=joint_marker[i], s=50, color=joint_colors[j])
                if j!=0:
                    axes[i].plot([pos[i][j][0], pos[i][j-1][0]], [pos[i][j][1], pos[i][j-1][1]], [pos[i][j][2], pos[i][j-1][2]], 'k')

    axes[0].set(xlabel='Z', ylabel='X', zlabel='Y', xlim=(-500, 500), ylim=(-500, 500), zlim=(-100, 900))
    axes[-1].legend(handles=
        [Line2D([],[], color=colors_R[j], linestyle='', marker='o', markersize=8, label=f"{chain_R[j]}") for j in range(len(chain_R))] + 
        [Line2D([],[], color=colors_L[j], linestyle='', marker='o', markersize=8, label=f"{chain_L[j]}") for j in range(3, len(chain_L))] + 
        [Line2D([],[], color='black', linestyle='-', label='Bones')],  loc='right', bbox_to_anchor=(1.4, 0.5))

# visualize_joint_locations()
# print_joint_properties(chain_R)
# print_joint_properties(chain_L)